In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.training_transformer import TrainingTransformer
from src.learning.training.label_collector import LabelCollector
from src.learning.training.training_file_reader import TrainingFileReader
from src.learning.models import create_cnn, create_mlp, create_multi_model

config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
transformer = TrainingTransformer(config)
collector = LabelCollector()

def memory_creator(new_frame, memory, length=4, interval=2):
    memory.append(new_frame)
    
    near_memory = memory[::-interval]
    if len(near_memory) < length:
        return None
    
    if len(memory) >= length * interval:
        memory.pop(0)
        
    return np.concatenate(near_memory, axis=2)

In [18]:
filename = 'lap_6_2020_01_24'

df = reader.read_telemetry_as_csv(filename + '.csv')
telemetry = collector.collect_numeric_inputs(df)
diffs = collector.collect_expert_labels(df)

frames = reader.read_training_video(filename + '.avi')
resized_frames_np = transformer.resize_video_for_training(frames)

memory = []
mem_frames = []
for frame in resized_frames_np:
    mem_frame = memory_creator(frame, memory)
    
    if mem_frame is not None:
        mem_frames.append(mem_frame)
        
resized_frames_np = np.array(mem_frames)

len_diff = telemetry.shape[0] - resized_frames_np.shape[0]
if len_diff > 0:
    telemetry = telemetry.iloc[len_diff:]
    diffs = diffs.iloc[len_diff:]
    
print(resized_frames_np.shape)
print(telemetry.shape)
print(telemetry.columns.values)
print(diffs.shape)
print(diffs.columns.values)

telemetry_np = telemetry.to_numpy()
diffs_np = diffs.to_numpy()

frames_train, frames_test, telemetry_train, telemetry_test, diffs_train, diffs_test = train_test_split(resized_frames_np, telemetry_np, diffs_np, test_size=0.2)

(6572, 40, 60, 12)
(6572, 4)
['cg' 'cs' 'ct' 'cb']
(6572, 4)
['d_gear' 'd_steering' 'd_throttle' 'd_braking']


In [19]:
mlp = create_mlp()
cnn = create_cnn()
multi = create_multi_model(mlp, cnn)
#multi.summary()

epochs = 6
batch_size = 32
verbose = 1

In [ ]:
multi.fit([telemetry_train, frames_train], diffs_train,
                validation_data=([telemetry_test, frames_test], diffs_test),
                epochs=epochs,
                batch_size=batch_size,
                verbose=verbose)